# MonkeyMadness

# 1.Enable GPU & SETUP ENVIROMENT
#### Deep learning algorithms are computationally intensive, and using a GPU can significantly reduce training times. Follow these steps to enable GPU acceleration:
##### Enabling GPU in Google Colab (Recomended)
1. Navigate to the top menu and click on "Runtime"
2. Select "Change runtime type"
3. In the dialog box:
    - Choose the appropriate runtime type (typically Python 3)
    - Set the "Hardware accelerator" to "GPU"
    - A Tesla T4 GPU is typically available for free users
4. Click "Save" to apply the changes
#### Local Machine Setup
- M1 chip or later (mac)
- CUDA already installed (Windows/Linux)
- Make sure to install req_local.txt in code block 2 "Setup Environment"
##### Performance: 30 epochs take ~20 minutes on T4 and might be faster on your own gpu

In [ ]:
import os #if os not available, add pip install
import torch #if not available on local, add !pip install torch before

current_dir = os.getcwd()


# Check if already in MonkeyMadness directory
if current_dir.endswith('MonkeyMadness'): #prevent nested cloning of repo
    print("✅ Already in MonkeyMadness directory, no need to clone again ✅")
else:
    # Safe cloning if not in the directory
    !git clone https://github.com/LiU-AI-Society/MonkeyMadness.git
    HOME = os.getcwd()
    print(f"Current working directory: {HOME}")
    %cd {HOME}/MonkeyMadness

    %pip install -r req_colab.txt  # Change to req_local.txt if on local machine, done with python version 3.12.1. make sure to restart the kernel after


from get_device import DeviceManager

# Get the device
device = DeviceManager.get_device()

# Print detailed device information
DeviceManager.print_device_info(device)

# Optional additional checks
if device.type == 'cuda':
    !nvidia-smi

Cloning into 'MonkeyMadness'...
remote: Enumerating objects: 1452, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 1452 (delta 79), reused 91 (delta 37), pack-reused 1305 (from 1)
Receiving objects: 100% (1452/1452), 579.57 MiB | 17.21 MiB/s, done.
Resolving deltas: 100% (80/80), done.
Updating files: 100% (1117/1117), done.
Current working directory: /content
/content/MonkeyMadness
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.2 MB/s eta 0:00:00
✅ Using CUDA GPU: Tesla T4 ✅
CUDA Device Name: Tesla T4
CUDA Device Count: 1
CUDA Current Device: 0
Wed Dec  4 16:48:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2    

# 2. CONVOLUTION BLOCK

This is just to get a feeling of the convolutional operation

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn.functional as F
from convolution import perform_convolution, plot_images

# Load the image
image_path = 'Monkey/training/training/n7/n7023.jpg'
image = Image.open(image_path).convert("RGB")

# Transform the image to a tensor
transform = transforms.ToTensor()
image_tensor = transform(image).unsqueeze(0)  # Add batch dimension

# Define vertical and horizontal line detection kernels
vertical_line_kernel = torch.tensor([[-1.0, 0.0, 1.0],
                                     [-1.0, 0.0, 1.0],
                                     [-1.0, 0.0, 1.0]], dtype=torch.float32)

horizontal_line_kernel = torch.tensor([[-1.0, -1.0, -1.0],
                                       [0.0,  0.0,  0.0],
                                       [1.0,  1.0,  1.0]], dtype=torch.float32)

# Perform the convolutions
horizontal_lines_image = perform_convolution(horizontal_line_kernel, image)
vertical_line_image = perform_convolution(vertical_line_kernel, image)

# plot the result

plot_images(image, vertical_line_image, horizontal_lines_image)

# 3. DATASET BLOCK
### You will use a monkey dataset consisting of ca 1000 images of monkey. The goal is to be able to classify them.
### There are ten monkey classes


In [ ]:
from torchvision import transforms, utils
from Dataset import CustomImageDataset, MonkeyImageDataset

NUM_OF_CLASSES = 10
IMAGE_SIZE = (64, 64) # DO NOT ALTER THIS PARAMETER

DATA_PERCENTAGE = 0.5
transform = transforms.Compose([
    #Randomly flip the images vertically
    #transforms.RandomVerticalFlip(p=0.2),  # Randomly flip the image vertically with 20% probability
    #transforms.RandomHorizontalFlip(p=0.2),  # Randomly flip the image horizontally with 20% probability
    #transforms.RandomRotation(degrees=15),  # Rotate the image randomly within a 15-degree range
    #transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Randomly change brightness, contrast, etc.
    transforms.ToTensor(),
    transforms.Resize((IMAGE_SIZE[0], IMAGE_SIZE[1])),

    #for imagenet
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
dataset = MonkeyImageDataset('Monkey/training/training', transform, NUM_OF_CLASSES, data_percentage = DATA_PERCENTAGE )
dataset.visualize(5)
#dataset.visualize_all_classes()


# 4. MODEL BLOCK
## Let's define our model!
### Lets build a Convolutional Neural Network (CNN). It uses convolutions (one can think of it as filters) to learn the different classes

In [ ]:
#MODEL CODE BLOCK

import torch.nn as nn
import torch.nn.functional as F
import torch
from torchsummary import summary


class MonkeyNET(nn.Module):
    def __init__(self, num_classes=10, input_size=(500, 500)):
        super(MonkeyNET, self).__init__()

        # First convolutional layer: 3 input channels (RGB),  output channels, kernel size 5, padding 2 to preserve size
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=16, kernel_size=3, padding=1)
        # Calculate the size of the fully connected layer dynamically
        self.fc_input_size = self._get_fc_input_size(input_size)
        self.fc = nn.Linear(self.fc_input_size, num_classes) # Adjusted for the final size after pooling


    def _get_fc_input_size(self, input_size):
        x = torch.zeros(1, 3, *input_size) # Create a dummy input tensor
        #x = self.attention(x)
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=(4, 4), stride=4)

        x = F.relu(self.conv2(x))

        x = F.max_pool2d(x, kernel_size=(8, 8), stride=8)


        return x.numel() # Total number of elements after conv layers

    def forward(self, x):
        # First conv -> ReLU -> Max Pooling
        x = F.relu(self.conv1(x))


        x = F.max_pool2d(x, kernel_size=(4, 4), stride=4)

        x = F.relu(self.conv2(x))

        x = F.max_pool2d(x, kernel_size=(8, 8), stride=8)




        # Flatten the tensor for fully connected layer
        x = x.view(x.size(0), -1)

        # Fully connected layer -> ReLU
        x = self.fc(x)

        return x

model = MonkeyNET(num_classes=NUM_OF_CLASSES, input_size=IMAGE_SIZE).to(device) #delete .to(device) if m1

summary(model, (3, IMAGE_SIZE[0], IMAGE_SIZE[1]))

# 5. TRAINING BLOCK

In [ ]:
from train import train, training_info
from test_model import test
import torch.optim
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torchvision.models as models
import datetime
from GradCam import GradCAM, generate_and_save_grad_cams
import numpy as np
current_datetime = datetime.datetime.now()



EPOCHS = 40 # how long should the model train
MODEL_NAME = "groupname and what you are testing"  #name for the training run to be able to compare


ID = f"{current_datetime.hour:02}:{current_datetime.minute:02}"
LR = 0.01   # how "much" should the model learn
BATCH_SIZE = 8  # how many images should the model see before updating

model_info = {
            'epochs' : EPOCHS,
            'batch_size' : BATCH_SIZE,
            'lr' : LR,
            'ID' : ID,
            'model_name' : MODEL_NAME
}

dataset_size = len(dataset)
train_size = int(0.8 * dataset_size)  # 80% for training
val_size = dataset_size - train_size   # 20% for validation
criterion = nn.CrossEntropyLoss() #This is the loss function

# Split the dataset
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders for training and validation sets
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=1, shuffle=False)

# Initialize the model
model = MonkeyNET(num_classes=NUM_OF_CLASSES, input_size=IMAGE_SIZE)

# Create optimizers for the model. This will try to find the optimal parameters in the model i.e this adjusts the model to improve the loss
optimizer = torch.optim.SGD(model.parameters(), lr=LR)

# Move models to gpu and train
model.to(device)
model, t_loss, t_acc, v_loss, v_acc = train(model, train_loader, val_loader, optimizer, criterion, device, start_epoch=0, num_epochs=EPOCHS, model_name=MODEL_NAME, unique_id=ID)


model_info['t_loss'] = t_loss
model_info['t_acc'] = t_acc

model_info['v_loss'] = v_loss
model_info['v_acc'] = v_acc

training_info(model_info=model_info)

# see metrics on the validation set

acc = test(model=model, testloader=val_loader, device=device, model_name=MODEL_NAME, unique_id=ID)



# 6.GradCAM block: The red parts are what the model "looks" at for making it's prediction

In [ ]:
# x-AI block

common_names = [
    "mantled_howler",
    "patas_monkey",
    "bald_uakari",
    "japanese_macaque",
    "pygmy_marmoset",
    "white_headed_capuchin",
    "silvery_marmoset",
    "common_squirrel_monkey",
    "black_headed_night_monkey",
    "nilgiri_langur"
]


grad_cam = GradCAM(model=model, target_layer=model.conv2) # change target layer to the last conv layer if you add more

generate_and_save_grad_cams(model, grad_cam, val_loader, common_names, save_dir="grads", device=device, index=np.random.randint(len(val_loader)))

# 7 COMPARING BLOCK
## See all performed experiments in graph to compare models

In [ ]:
import matplotlib.pyplot as plt
from train import plot_experiments

plot_experiments('training_metrics')

# Send your best model to us for us to run it on a test set!

## Upload to google drive folder